In [1]:
import numpy as np
import torch
from longvu.builder import load_pretrained_model
from longvu.constants import (
    DEFAULT_IMAGE_TOKEN,
    IMAGE_TOKEN_INDEX,
)
from longvu.conversation import conv_templates, SeparatorStyle
from longvu.mm_datautils import (
    KeywordsStoppingCriteria,
    process_images,
    tokenizer_image_token,
)
from decord import cpu, VideoReader

tokenizer, model, image_processor, context_len = load_pretrained_model(
    "./checkpoints/LongVU_Llama3_2_3B", None, "cambrian_llama",load_8bit=True
)

model.eval()
video_path = "./examples/video2.mp4"
qs = "What's the prompt to generate the video?"

vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
fps = float(vr.get_avg_fps())
frame_indices = np.array([i for i in range(0, len(vr), round(fps),)])
video = []
for frame_index in frame_indices:
    img = vr[frame_index].asnumpy()
    video.append(img)
video = np.stack(video)
image_sizes = [video[0].shape[:2]]
video = process_images(video, image_processor, model.config)
video = [item.unsqueeze(0) for item in video]

qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
conv = conv_templates["llama3"].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=video,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0.2,
        max_new_tokens=128,
        use_cache=True,
        stopping_criteria=[stopping_criteria],
    )
pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(f"\nAnswer: {pred}") # 4bit -> 11G GPU ; 8 bit -> 12G GPU 视频长度如果是5秒，则需要9G GPU


/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading Cambrian from ./checkpoints/LongVU_Llama3_2_3B


OSError: Incorrect path_or_model_id: './checkpoints/LongVU_Llama3_2_3B'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [2]:
import numpy as np
import torch
from longvu.builder import load_pretrained_model
from longvu.constants import (
    DEFAULT_IMAGE_TOKEN,
    IMAGE_TOKEN_INDEX,
)
from longvu.conversation import conv_templates, SeparatorStyle
from longvu.mm_datautils import (
    KeywordsStoppingCriteria,
    process_images,
    tokenizer_image_token,
)
from decord import cpu, VideoReader

tokenizer, model, image_processor, context_len = load_pretrained_model(
    "./checkpoints/LongVU_Qwen2_7B", None, "cambrian_qwen",load_4bit=True
)

model.eval()
video_path = "./examples/video1.mp4"
qs = "Describe this video in detail"

vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
fps = float(vr.get_avg_fps())
frame_indices = np.array([i for i in range(0, len(vr), round(fps),)])
video = []
for frame_index in frame_indices:
    img = vr[frame_index].asnumpy()
    video.append(img)
video = np.stack(video)
image_sizes = [video[0].shape[:2]]
video = process_images(video, image_processor, model.config)
video = [item.unsqueeze(0) for item in video]

qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
conv = conv_templates["qwen"].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=video,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0.2,
        max_new_tokens=128,
        use_cache=True,
        stopping_criteria=[stopping_criteria],
    )
pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

print(f"\nAnswer: {pred}") #17G GPU

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos t


Answer: The video begins with a scene featuring two characters in a fantastical setting, where one character, dressed in a bright yellow and red outfit with a mask, is seen in a dynamic pose suggesting movement or action, while the other character, an older figure with a white beard and a blue robe, stands with a contemplative expression. The background is filled with green foliage and a mountainous landscape, indicating an outdoor setting.

As the video progresses, the character in the yellow and red outfit is seen in a more dynamic pose, with one arm raised, possibly signaling a gesture or movement, while the older character's expression becomes more concerned or surprised


In [1]:
import numpy as np
import torch
from longvu.builder import load_pretrained_model
from longvu.constants import (
    DEFAULT_IMAGE_TOKEN,
    IMAGE_TOKEN_INDEX,
)
from longvu.conversation import conv_templates, SeparatorStyle
from longvu.mm_datautils import (
    KeywordsStoppingCriteria,
    process_images,
    tokenizer_image_token,
)
from decord import cpu, VideoReader

tokenizer, model, image_processor, context_len = load_pretrained_model(
    "./checkpoints/LongVU_Llama3_2_1B", None, "cambrian_llama",load_8bit=True
)

model.eval()
video_path = "./examples/video1.mp4"
qs = "What's the prompt to generate the video?"

vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
fps = float(vr.get_avg_fps())
frame_indices = np.array([i for i in range(0, len(vr), round(fps),)])
video = []
for frame_index in frame_indices:
    img = vr[frame_index].asnumpy()
    video.append(img)
video = np.stack(video)
image_sizes = [video[0].shape[:2]]
video = process_images(video, image_processor, model.config)
video = [item.unsqueeze(0) for item in video]

qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
conv = conv_templates["llama3"].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=video,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0.2,
        max_new_tokens=128,
        use_cache=True,
        stopping_criteria=[stopping_criteria],
    )
pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(f"\nAnswer: {pred}") 

/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading Cambrian from ./checkpoints/LongVU_Llama3_2_1B


/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_s


Answer: The prompt to generate the video is "a man in a yellow costume and a man in a blue outfit".


In [1]:
import numpy as np
import torch
import gc
from longvu.builder import load_pretrained_model
from longvu.constants import (
    DEFAULT_IMAGE_TOKEN,
    IMAGE_TOKEN_INDEX,
)
from longvu.conversation import conv_templates, SeparatorStyle
from longvu.mm_datautils import (
    KeywordsStoppingCriteria,
    process_images,
    tokenizer_image_token,
)
from decord import cpu, VideoReader

model_path = "new_checkpoints/cambrian_llama3_2_1B_25epoches/checkpoint-6000"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path, None, "cambrian_llama"
)

model.eval()
video_path = "/home/fenghe/vidprom/cog_videos_example/cog-de3c22e1-6a93-5971-b09b-fc976ddd7803.mp4"
qs = "What's the prompt to generate the video?"

vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
fps = float(vr.get_avg_fps())
frame_indices = np.array([i for i in range(0, len(vr), round(fps),)])
video = []
for frame_index in frame_indices:
    img = vr[frame_index].asnumpy()
    video.append(img)
video = np.stack(video)
image_sizes = [video[0].shape[:2]]
video = process_images(video, image_processor, model.config)
video = [item.unsqueeze(0) for item in video]

qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
conv = conv_templates["llama3"].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=video,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0.2,
        max_new_tokens=128,
        use_cache=True,
        stopping_criteria=[stopping_criteria],
    )
pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(f"Model: {model_path}\nAnswer: {pred}") 

with open('video_inversion_answer.txt', 'a') as f:
    f.write(f"Model: {model_path}\nAnswer: {pred}\n\n")

if 'model' in globals():
    del model
    gc.collect()
    torch.cuda.empty_cache()

/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading Cambrian from new_checkpoints/cambrian_llama3_2_1B_25epoches/checkpoint-6000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for

Model: new_checkpoints/cambrian_llama3_2_1B_25epoches/checkpoint-6000
Answer: a bird with a blue face and orange beak


In [1]:
import numpy as np
import torch
import gc
from longvu.builder import load_pretrained_model
from longvu.constants import (
    DEFAULT_IMAGE_TOKEN,
    IMAGE_TOKEN_INDEX,
)
from longvu.conversation import conv_templates, SeparatorStyle
from longvu.mm_datautils import (
    KeywordsStoppingCriteria,
    process_images,
    tokenizer_image_token,
)
from decord import cpu, VideoReader

model_path = "new_checkpoints/cambrian_llama3_2_1B_35epoches/checkpoint-35000"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path, None, "cambrian_llama"
)

model.eval()
video_path = "/home/fenghe/vidprom/cog_videos_example/cog-0010fb2b-f6cd-51af-a0b6-b38812f965ea.mp4"
qs = "What's the prompt to generate the video?"

vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
fps = float(vr.get_avg_fps())
frame_indices = np.array([i for i in range(0, len(vr), round(fps),)])
video = []
for frame_index in frame_indices:
    img = vr[frame_index].asnumpy()
    video.append(img)
video = np.stack(video)
image_sizes = [video[0].shape[:2]]
video = process_images(video, image_processor, model.config)
video = [item.unsqueeze(0) for item in video]

qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
conv = conv_templates["llama3"].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=video,
        image_sizes=image_sizes,
        do_sample=False,
        temperature=0.2,
        max_new_tokens=128,
        use_cache=True,
        stopping_criteria=[stopping_criteria],
    )
pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(f"Model: {model_path}\nAnswer: {pred}") 

with open('video_inversion_answer.txt', 'a') as f:
    f.write(f"Model: {model_path}\nAnswer: {pred}\n\n")

if 'model' in globals():
    del model
    gc.collect()
    torch.cuda.empty_cache()

/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading Cambrian from new_checkpoints/cambrian_llama3_2_1B_35epoches/checkpoint-35000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/fenghe/anaconda3/envs/longvu/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model: new_checkpoints/cambrian_llama3_2_1B_35epoches/checkpoint-35000
Answer: a woman with long hair and a white dress


In [5]:
import json
import pandas as pd

# 读取video_prompts.json
with open('video_prompts.json', 'r') as f:
    prompts = json.load(f)

# 将prompts转换为DataFrame
df = pd.DataFrame(prompts)


In [10]:
# 划分训练集和测试集
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# 保存训练集和测试集，保留原来json的格式
with open('train_prompts_90percent.json', 'w') as f:
    json.dump(train_df.to_dict(orient='records'), f, indent=2)

with open('test_prompts_10percent.json', 'w') as f:
    json.dump(test_df.to_dict(orient='records'), f, indent=2)



In [1]:
import torch
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video
import gc



pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-2b",
    torch_dtype=torch.float16
)

# pipe.enable_model_cpu_offload()
# pipe.enable_sequential_cpu_offload()
pipe.to("cuda") 
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()


/home/fenghe/anaconda3/envs/cogvid_2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-24 23:19:35,881] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/fenghe/anaconda3/envs/cogvid_2/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/fenghe/anaconda3/envs/cogvid_2/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/fenghe/anaconda3/envs/cogvid_2/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/fenghe/anaconda3/envs/cogvid_2/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/fenghe/anaconda3/envs/cogvid_2/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/fenghe/anaconda3/envs/cogvid_2/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_erro

In [2]:

prompt = "a purple car driving on the highway"
video = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=50,
    num_frames=49,
    guidance_scale=6,
    generator=torch.Generator(device="cuda").manual_seed(66),
).frames[0]

export_to_video(video, "output/specific_examples/output_8.mp4", fps=8)

if 'pipe' in globals():
    del video
    gc.collect()
    torch.cuda.empty_cache()


100%|██████████| 50/50 [01:25<00:00,  1.70s/it]


In [ ]:
# To get started, PytorchAO needs to be installed from the GitHub source and PyTorch Nightly.
# Source and nightly installation is only required until next release.

import torch
from diffusers import AutoencoderKLCogVideoX, CogVideoXTransformer3DModel, CogVideoXPipeline
from diffusers.utils import export_to_video
# from transformers import T5EncoderModel
# from torchao.quantization import quantize_, int8_weight_only, int8_dynamic_activation_int8_weight

# quantization = int8_weight_only

# text_encoder = T5EncoderModel.from_pretrained("THUDM/CogVideoX-5b", subfolder="text_encoder", torch_dtype=torch.bfloat16)
# quantize_(text_encoder, quantization())

# transformer = CogVideoXTransformer3DModel.from_pretrained("THUDM/CogVideoX-5b", subfolder="transformer", torch_dtype=torch.bfloat16)
# quantize_(transformer, quantization())

# vae = AutoencoderKLCogVideoX.from_pretrained("THUDM/CogVideoX-2b", subfolder="vae", torch_dtype=torch.bfloat16)
# quantize_(vae, quantization())

# Create pipeline and run inference
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-2b",
    # text_encoder=text_encoder,
    # transformer=transformer,
    # vae=vae,
    torch_dtype=torch.bfloat16,
)
pipe.to("cuda")
pipe.enable_model_cpu_offload()
pipe.vae.enable_tiling()


Loading pipeline components...:   0%|          | 0/5 [02:48<?, ?it/s]


In [ ]:

prompt = "A woman is smoking a cigarette while sitting on a bed."

video = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=50,
    num_frames=49,
    guidance_scale=6,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

export_to_video(video, "output.mp4", fps=8)
